In [6]:
import pandas as pd
import numpy as np
import re

In [16]:
path = "../Data/icd_mexico.pkl"
df = pd.read_pickle(path)[['des_causa_a','causa_a']]

for x in df.columns:
  df[x] = df[x].astype(str).str.lower()

In [24]:
df[df.causa_a.apply(lambda x: len(x.split()) > 1)]

,des_causa_a,causa_a
1,herida producida por proyectil disparado por arma de fuego penetrante de cr,s019 x958
2,decapitacion,s18x y097
11,herida producida por proyectil disparado por arma de fuego penetrante en cr,s019 x958
13,decapitado,s18x y097
14,indeterminada,r99x y098
...,...,...
606140,debido a las carencias de viceras toracicas abdominales y pelvicas no-es posible concluir con respecto a la causa de muerte.,r688 r688
606145,hemorragia severa de capas menignas y frac turas de calotas-y craneo base de piso de loa nterios provocado por proyectirl de arma de fuego,s018 s018
606153,anoxia anoxica secundaria a axficia mecanica por estrangulacion-en su modalidad armada,t71x t71x
606161,shock hiporvolemico con hemorragia masiva de capas meningues y masa encefalica secundario a-a traumatismo directo y profundo a cavidad craneal.,s068 s069


In [8]:
pd.set_option('display.max_colwidth',600)
df.reset_index(drop=True).loc[8933676]

des_causa_a    hemorragia   masiva   interna   y externa  aguda    severa   e irreversible   de organos   hueco-e izquiwerdo   todo  esto secundario  a herida   producidas  por proyectil   de arma  de fuegos
causa_a                                                                                                                                                                                               r578 t141
Name: 8933676, dtype: object

In [9]:
def read_catalogue(row):
  columnas = ['CATALOG_KEY', 'DIA_NOMBRE']
  df = pd.read_excel(path, usecols = columnas)
  df = df.apply(lambda x: x.str.lower())
  df.columns = df.columns.str.lower()
  df = df.rename(
    columns = {'catalog_key':'cie','dia_nombre':'descripcion'})
  df = df[~df.cie.isin(df.cie.str[:3])]
  return df

path = '/Users/pierredelice/Library/CloudStorage/Dropbox/Mac/Documents/GitHub/EM_FellegiSunter/data/CATALOGO_CIE_10_PRODUCCION_29042022.xlsx'
catalogue = read_catalogue(path)


In [10]:
catalogue = list(set(catalogue[catalogue['cie'].isin(df['causa_a'].value_counts().index)].descripcion))

In [11]:

# Example dataframe for diagnostics
data = {
    'des_causa_a': [
        'indeterminada',
        'herida producida por proyectil disparado por arma de fuego',
        'decapitacion',
        'choque septico',
        'sindrome hepatorrenal',
        'hemorragia masiva interna y externa aguda',
        'traumatismo craneoencefalico severo',
        'hemorragia cerebral',
        'fractura de vertebras cervicales',
        'choque hipovolemico'
    ],
    'causa_a': [
        'r99x',
        's019 x958',
        's18x y097',
        'r572',
        'k767',
        'r578 t141',
        'x599',
        'i619',
        's129',
        'r571'
    ]
}

df = pd.DataFrame(data)

# Function to match catalogue events in diagnostics
def extract_event(diagnostic, catalogue):
    for event in catalogue:
        # Exact match
        if event.lower() in diagnostic.lower():
            return event
        # Partial match: checking if words in the event appear in the diagnostic
        event_words = event.lower().split()
        if all(word in diagnostic.lower() for word in event_words if len(word) > 3):  # Filter out short common words
            return event
    return "No match"

# Apply the function to extract events from the diagnostics
df['extracted_event'] = df['des_causa_a'].apply(lambda x: extract_event(x, catalogue))


df[['des_causa_a', 'causa_a', 'extracted_event']]


,des_causa_a,causa_a,extracted_event
0,indeterminada,r99x,tos
1,herida producida por proyectil disparado por arma de fuego,s019 x958,tos
2,decapitacion,s18x y097,tos
3,choque septico,r572,tos
4,sindrome hepatorrenal,k767,tos
5,hemorragia masiva interna y externa aguda,r578 t141,tos
6,traumatismo craneoencefalico severo,x599,tos
7,hemorragia cerebral,i619,tos
8,fractura de vertebras cervicales,s129,tos
9,choque hipovolemico,r571,hipo


In [14]:
import spacy
from spacy import displacy

# Load a pre-trained NER model (replace with a medical-specific model if available)
nlp = spacy.load("en_core_web_sm")  # Replace with "en_core_sci_md" for SciSpaCy or other specialized models

# Example diagnostic descriptions
diagnostics = [
    "choque septico",
    "hemorragia masiva interna y externa aguda",
    "traumatismo craneoencefalico severo",
    "hemorragia   masiva   interna   y externa  aguda    severa   e irreversible   de organos   hueco-e izquiwerdo   todo  esto secundario  a herida   producidas  por proyectil   de arma  de fuegos"
]

# Process each diagnostic description
for diagnostic in diagnostics:
    doc = nlp(diagnostic)
    print(f"Diagnostic: {diagnostic}")
    for ent in doc.ents:
        print(f"  - Entity: {ent.text}, Label: {ent.label_}")
    print("\n")

# Visualization of entities (optional)
displacy.render(doc, style="ent", jupyter=True)


Diagnostic: choque septico


Diagnostic: hemorragia masiva interna y externa aguda
  - Entity: hemorragia, Label: ORG


Diagnostic: traumatismo craneoencefalico severo


Diagnostic: hemorragia   masiva   interna   y externa  aguda    severa   e irreversible   de organos   hueco-e izquiwerdo   todo  esto secundario  a herida   producidas  por proyectil   de arma  de fuegos
  - Entity: hemorragia, Label: ORG
  - Entity: interna, Label: PERSON




In [15]:
from transformers import pipeline

# Load NER pipeline
nlp = pipeline("ner", model="dslim/bert-base-NER")  # Replace with a medical-specific model if needed

# Example diagnostic descriptions
diagnostics = [
    "choque septico y ",
    "hemorragia masiva interna y externa aguda",
    "traumatismo craneoencefalico severo",
    "hemorragia   masiva   interna   y externa  aguda    severa   e irreversible   de organos   hueco-e izquiwerdo   todo  esto secundario  a herida   producidas  por proyectil   de arma  de fuegos"
]

# Extract entities
for diagnostic in diagnostics:
    print(f"Diagnostic: {diagnostic}")
    entities = nlp(diagnostic)
    for entity in entities:
        print(f"  - Entity: {entity['word']}, Label: {entity['entity']}")
    print("\n")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Diagnostic: choque septico y 


Diagnostic: hemorragia masiva interna y externa aguda


Diagnostic: traumatismo craneoencefalico severo


Diagnostic: hemorragia   masiva   interna   y externa  aguda    severa   e irreversible   de organos   hueco-e izquiwerdo   todo  esto secundario  a herida   producidas  por proyectil   de arma  de fuegos
  - Entity: f, Label: I-LOC


